In [4]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [5]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_thunder_3.tflite")
interpreter.allocate_tensors()

In [6]:
lowerjoints_list = [13,14,15,16]

In [8]:
def locating_joints(frame, keypoints, confidence_threshold):
    #y, x, c = frame.shape
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [9]:
def knee_distance(frame,point):
   # y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [10]:
def left_slope(frame,point):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [11]:
def ankle_distance(frame,point):
    #y, x, c = frame.shape
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [12]:
def right_slope(frame,point):
  import math
  #y, x, c = frame.shape
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [13]:
EDGES = {
    (13, 15): 'm',
    #(12, 14): 'c',
    (14, 16): 'c',
    (13, 14): 'c',
    (16,15) : 'm'
}

In [14]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    #y, x, c = frame.shape
    print(keypoints.shape)

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = joints_coordinates[p1]
        y2, x2, c2 = joints_coordinates[p2]
        
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [15]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  right_m= (y4-y2)/(x4-x2)
  left_m = (y3-y1)/(x3-x1)
  print("knee:", dis)
  print("ankle:", ankle_dis)
  print("left",left_m)
  print("Right",right_m)
#8739.635093408378
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  ankle = (x4-x3)**2+(y4-y3)**2
  #dis below 88 and ankle below 88, dis >80 and ankle > 80
  if (dis <= 7195.855191894665 and dis > 5608.360385571377 and ankle_dis <= 7195.855191894665 and ankle_dis > 5608.855051436771) or (dis < 7195.855191894665 and dis > 6495.855191894665 and ankle_dis >= 7195.855191894665 and ankle_dis <= 7795.855191894665 ):
     print("running loop 1")
     cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 5608.855051436771 and ankle_dis <= 5608.855051436771) or (dis < 5608.855051436771 and ankle_dis > 5608.855051436771 and ankle_dis <= 5908.855051436771):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis >= 7195.855191894665 and ankle_dis >= 7195.855191894665):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
     cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     pass
            
       
        
        

        
    
    

In [16]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0),256,256)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("hello boy")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("gurs")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("Kaur")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    # Rendering
    locating_joints(frame, keypoints_with_scores, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame,keypoints_with_scores)
    ankle_distance(frame,keypoints_with_scores)
    left_slope(frame,keypoints_with_scores)
    right_slope(frame,keypoints_with_scores)
    insoles(frame)
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
hello boy
[[[[0.28930885 0.5035907  0.02606222]
   [0.23493655 0.5448648  0.01056221]
   [0.25078943 0.5266622  0.0130116 ]
   [0.128446   0.84680855 0.05682543]
   [0.12678084 0.7576686  0.08745021]
   [0.21525371 0.46871114 0.1769318 ]
   [0.25528136 0.79920065 0.02429789]
   [0.74243486 0.12623684 0.15428856]
   [0.64661425 0.6810959  0.05196214]
   [0.83066094 0.0144909  0.06734684]
   [0.8485818  0.45478582 0.02877316]
   [0.83709544 0.60546446 0.08271307]
   [0.80158883 0.80490017 0.01295811]
   [0.85201573 0.44233602 0.1071589 ]
   [0.8625392  0.44699034 0.08308509]
   [0.846861   0.4639976  0.01907966]
   [0.85743624 0.45575196 0.02982

In [28]:
cap.release()
cv2.destroyAllWindows()